In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 910.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Import libraries
import os
import streamlit as st
from pyngrok import ngrok

In [ ]:
# Set up Ngrok tunnel (Replace with your actual token)
NGROK_AUTH_TOKEN = "<YOUR_NGROK_AUTH_TOKEN_HERE>"

# Replace the line above with your own Ngrok authentication token.
# To get your token, sign up at https://ngrok.com and go to the "Auth" section.

!ngrok authtoken $NGROK_AUTH_TOKEN

# Kill any existing tunnels (Optional, if needed)
os.system("pkill -f ngrok")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


15

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
import cv2

# Load model
model = tf.keras.models.load_model("/content/drive/MyDrive/inception_model.keras")

# Function to preprocess image
def preprocess_image(img):
    img = cv2.imdecode(np.frombuffer(img.read(), np.uint8), 1)
    img = cv2.resize(img, (299, 299))  # InceptionV3 input size
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Streamlit UI
st.title("🩺 PneumoScan - Pneumonia Detection")

uploaded_file = st.file_uploader("Upload a Chest X-Ray", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded X-Ray", use_container_width=True)

    # Preprocess and predict
    img = preprocess_image(uploaded_file)
    prediction = model.predict(img)[0][0]  # Assuming binary classification (0 = Normal, 1 = Pneumonia)

    if prediction > 0.5:
        st.error("🔴 **Pneumonia Detected** (Confidence: {:.2f}%)".format(prediction * 100))
    else:
        st.success("🟢 **Normal Lungs** (Confidence: {:.2f}%)".format((1 - prediction) * 100))


Writing app.py


In [ ]:
# Start Streamlit app in background
os.system("streamlit run app.py &")

# Create Ngrok tunnel and print the public link
public_url = ngrok.connect(8501).public_url
print(f"Your Streamlit app is live at: {public_url}")


Your Streamlit app is live at: https://427d-34-72-11-90.ngrok-free.app
